In [80]:
from models.SCGAN import SCGAN
from SCDataset.SCDataset import SCDataLoader
from models.models import create_model



In [85]:
class Options:
    def __init__(self,
                 phase = "train",
                 beta1 = 0.5,
                 beta2 = 0.999,
                 g_lr = 2e-4,
                 d_lr = 2e-4,
                 lambda_A = 10.0,
                 lambda_B = 10.0,
                 lambda_idt = 0.5,
                 lambda_his_lip=1.0,
                 lambda_his_skin=0.1,
                 lambda_his_eye=1.0,
                 lambda_vgg=5e-3,
                 num_epochs=100,
                 epochs_decay=0,
                 g_step=1,
                 log_step=8,
                 save_step=2048,
                 snapshot_path="./checkpoints/",
                 save_path="./results/",
                 snapshot_step=10,
                 perceptual_layers=3,
                 partial=False,
                 interpolation=False,
                 init_type="xavier",
                 dataroot="MT-Dataset/images",
                 dirmap="MT-Dataset/parsing",
                 batchSize=1,
                 input_nc=3,
                 img_size=256,
                 output_nc=3,
                 d_conv_dim=64,
                 d_repeat_num=3,
                 ngf=64,
                 gpu_ids="0",
                 nThreads=2,
                 norm1="SN",
                 serial_batches=True,
                 n_components=3,
                 n_res=3,
                 padding_type="reflect",
                 use_flip=0,
                 n_downsampling=2,
                 style_dim=192,
                 mlp_dim=256):
        self.phase = phase
        self.beta1 = beta1
        self.beta2 = beta2
        self.g_lr = g_lr
        self.d_lr = d_lr
        self.lambda_A = lambda_A
        self.lambda_B = lambda_B
        self.lambda_idt = lambda_idt
        self.lambda_his_lip = lambda_his_lip
        self.lambda_his_skin = lambda_his_skin
        self.lambda_his_eye = lambda_his_eye
        self.lambda_vgg = lambda_vgg
        self.num_epochs = num_epochs
        self.epochs_decay = epochs_decay
        self.g_step = g_step
        self.log_step = log_step
        self.save_step = save_step
        self.snapshot_path = snapshot_path
        self.save_path = save_path
        self.snapshot_step = snapshot_step
        self.perceptual_layers = perceptual_layers
        self.partial = partial
        self.interpolation = interpolation

        self.init_type = init_type
        self.dataroot = dataroot
        self.dirmap = dirmap
        self.batchSize = batchSize
        self.input_nc = input_nc
        self.img_size = img_size
        self.output_nc = output_nc
        self.d_conv_dim = d_conv_dim
        self.d_repeat_num = d_repeat_num
        self.ngf = ngf
        self.gpu_ids = gpu_ids
        self.nThreads = nThreads
        self.norm1 = norm1
        self.serial_batches = serial_batches
        self.n_components = n_components
        self.n_res = n_res
        self.padding_type = padding_type
        self.use_flip = use_flip
        self.n_downsampling = n_downsampling
        self.style_dim = style_dim
        self.mlp_dim = mlp_dim

In [86]:
opt = Options(phase="test")

In [87]:
# dataloader = SCDataLoader(opt)
# SCGan = create_model(opt, dataloader)
# SCGan.test()